In [209]:
from scipy import signal
import numpy as np
import librosa
import sofa
import soundfile as sf
from IPython.display import Audio
import sys,glob
import scipy.stats
from spafe.features.gfcc import gfcc
#from binaural_cues import *

In [210]:
size = 44100
WhiteNoise = np.random.normal(0, 1, size=size)
sofaDir = 'dane_hrtf/sofa/*.sofa'
_SOFA = glob.glob(sofaDir)
HRTFs = list([sofa.Database.open(_SOFA[x]) for x in range(0, 30)])
fs_H = HRTFs[1].Data.SamplingRate.get_values()[0]
positions = HRTFs[1].Source.Position.get_values(system='spherical')
HRTFs[1].Dimensions.dump()
mix = np.vstack([WhiteNoise, WhiteNoise])

I: 1
C: 3
R: 2
E: 1
N: 256
M: 2304
S: 0


In [211]:
def Efficient_ccf(x1,x2):
    """calculate cross-crrelation function in frequency domain, which is more 
    efficient than the direct calculation"""
    
    if x1.shape[0] != x2.shape[0]:
        raise Exception('length mismatch')
    wav_len = x1.shape[0]
    # hanning window before fft
    wf = np.hanning(wav_len)
    x1 = x1*wf
    x2 = x2*wf

    X1 = np.fft.fft(x1,2*wav_len-1)# equivalent to add zeros 
    X2 = np.fft.fft(x2,2*wav_len-1)
    ccf_unshift = np.real(np.fft.ifft(np.multiply(X1,np.conjugate(X2))))
    ccf = np.concatenate([ccf_unshift[wav_len:],ccf_unshift[:wav_len]],axis=0)
    
    return ccf

In [212]:
def get_ITD(x,fs,max_delay=None,inter_method='parabolic'):
    """
    estimate ITD based on interaural corss-correlation function
    itd = chann0_delay - chann1_delay
    corr(i) = sum(x0[t]*x1[t-i])
        | >0 chann1 lead
    itd |
        | <0 chann1 lead
    input: 
        max_delay: maximum value of ITD, default value: 1ms
        inter_method: method of ccf interpolation, "None"(default),"parabolic",'exponential'.
    """
    wav_len = x.shape[0]
    
    # detrend
    # x_detrend = x-np.mean(x,axis=0)
    x_detrend = x
    
    if max_delay == None:
        max_delay = int(1e-3*fs)
    
    if False:
        # time domain
        ccf_full = np.correlate(x_detrend[:,0],x_detrend[:,1],mode='full')
        ccf = ccf_full[wav_len-1-max_delay:wav_len+max_delay]
    else:
        # frequency domain
        ccf_full = Efficient_ccf(x_detrend[:,0],x_detrend[:,1])
        ccf = ccf_full[wav_len-1-max_delay:wav_len+max_delay]
    
    ccf_std = ccf/(np.sqrt(np.sum(x_detrend[:,0]**2)*np.sum(x_detrend[:,1]**2)))
    max_pos = np.argmax(ccf)
    
    ######################
    if False:
        plt.figure(1)
        plt.clf()
        plt.subplot(311);    plt.plot(ccf_std);
        plt.plot([wav_len-max_delay-1,wav_len-max_delay-1],[0,1],'r')
        plt.plot([wav_len+max_delay-1,wav_len+max_delay-1],[0,1],'r')
        plt.plot(wav_len-1-max_delay+max_pos,ccf_std[wav_len-1-max_delay+max_pos],'x',linewidth=2)

        plt.subplot(312);    plt.plot(x[:,0])
        plt.subplot(313);    plt.plot(x[:,1])

        plt.show(block=False)
        plt.pause(0.001)
    ######################
    
    # exponential interpolation 
    delta = 0
    if inter_method == 'exponential':
        if max_pos> 0 and max_pos < max_delay*2-2:
            if np.min(ccf[max_pos-1:max_pos+2]) > 0:
                delta = (np.log(ccf[max_pos+1])-np.log(ccf[max_pos-1]))/\
                            (4*np.log(ccf[max_pos])-
                             2*np.log(ccf[max_pos-1])-
                             2*np.log(ccf[max_pos+1]))
    elif inter_method == 'parabolic':
        if max_pos> 0 and max_pos < max_delay*2-2:
            delta = (ccf[max_pos-1]-ccf[max_pos+1])/(2*(ccf[max_pos+1]-2*ccf[max_pos]+ccf[max_pos-1]))
        
    ITD = float((max_pos-max_delay-1+delta))/fs*1e3

    return [ITD,ccf_std]

In [213]:
def parameters(array, sr):
    params = []
    for i in range(2):
        params.append(np.mean(array[i]))
        params.append(np.std(array[i]))
        params.append(np.median(array[i]))
        params.append(np.percentile(array[i], 25))
        params.append(np.percentile(array[i], 75))
        params.append(scipy.stats.iqr(array[i], rng=(10, 90)))
        params.append(scipy.stats.kurtosis(array[i]))
        params.append(scipy.stats.skew(array[i]))
        params.append(np.min(array[i]))
        params.append(np.max(array[i]))
        
        params.append(np.mean(librosa.feature.spectral_centroid(array[i], sr=sr)))
        params.append(np.mean(librosa.feature.spectral_rolloff(array[i], sr=sr)))
        
        params.extend(librosa.feature.mfcc(array[i], sr=sr, n_mfcc=13).flatten())
        params.extend(gfcc(array[i], fs=sr, num_ceps=13).flatten())
        #params.append(librosa.feature.zero_crossing_rate(array[i], frame_length=sr).flatten())
        params.append(librosa.feature.rms(array[i], frame_length=sr).flatten())
    length = len(params)
    idx = length//2
    l = params[:idx]
    r = params[idx:]
    for i,j in zip(l,r):
        params.append(np.mean([i,j]))
        params.append(np.std([i,j]))
    params.append(10*np.log10(np.sum(array[:,1]**2)/np.sum(array[:,0]**2)+1e-10))
    params.append(get_ITD(array, sr)[0])
    return params 

a = np.array(parameters(mix, 44100))
print(a)

[0.0015897102192726949 0.9996962228815957 0.004147424054175357 ...
 0.08507712351212651 2.5894787211811536 -1.0204081632653061]


C:\Users\Jakub Bociek\AppData\Local\Temp\ipykernel_8868\1004166621.py:15: FutureWarning: Pass y=[ 0.70914928  0.955458   -1.51976846 ...  1.52300419  1.20179133
  1.51160428] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  params.append(np.mean(librosa.feature.spectral_centroid(array[i], sr=sr)))
C:\Users\Jakub Bociek\AppData\Local\Temp\ipykernel_8868\1004166621.py:16: FutureWarning: Pass y=[ 0.70914928  0.955458   -1.51976846 ...  1.52300419  1.20179133
  1.51160428] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  params.append(np.mean(librosa.feature.spectral_rolloff(array[i], sr=sr)))
C:\Users\Jakub Bociek\AppData\Local\Temp\ipykernel_8868\1004166621.py:18: FutureWarning: Pass y=[ 0.70914928  0.955458   -1.51976846 ...  1.52300419  1.20179133
  1.51160428] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  params.extend(librosa.feature

TypeError: Mismatch between array dtype ('object') and format specifier ('%.18e')